<a href="https://colab.research.google.com/github/hhhhh0102/ner_project/blob/main/ner_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install transformers

In [54]:
!pip install sentencepiece

In [55]:
!git clone https://github.com/hhhhh0102/ner_project.git

fatal: destination path 'ner_project' already exists and is not an empty directory.


In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
import os
import sys
sys.path.append("/content/ner_project/ner_model")

In [58]:
import numpy as np
import tensorflow as tf
from Preprocess import preprocess
from make_model import make_model
from tokenizer import KoBertTokenizer

def ner_task(textpath):
    tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
    model = tf.keras.models.load_model("/content/drive/MyDrive/my_model.h5")
    my_model = make_model()
    my_model.model_input.set_dataset("/content/ner_project/ner_model/train_data")
    my_model.model_input.make_input()
    line_list = []

    f = open(textpath, mode='r', encoding='utf-8')
    while True:
        line = f.readline()
        if not line: break
        line_list.append(line)
    f.close()

    f = open('ner_result.txt', mode='wt', encoding='utf-8')
    for l in list(range(len(line_list))):

        tokenized_sentence = np.array([tokenizer.encode(line_list[l], max_length=88, pad_to_max_length=True)])
        tokenized_mask = np.array([[int(x != 1) for x in tokenized_sentence[0].tolist()]])
        ans = model.predict([tokenized_sentence, tokenized_mask])
        ans = np.argmax(ans, axis=2)

        tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, ans[0]):

            if (token.startswith("▁")):
                new_labels.append(my_model.model_input.pr.index_to_ner[label_idx])
                new_tokens.append(token[1:])
            elif (token == '[CLS]'):
                pass
            elif (token == '[SEP]'):
                pass
            elif (token == '[PAD]'):
                pass
            elif (token != '[CLS]' or token != '[SEP]'):
                new_tokens[-1] = new_tokens[-1] + token

        for ll in list(range(len(new_tokens))):
            f.write("{}\t{}\t{}\n".format(ll + 1, new_tokens[ll], new_labels[ll]))
        f.write("\n")

    f.close()

In [59]:
ner_task("/content/ner_project/pre_trained_data.txt")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
